# Undersampling

In this notebook we'll explain how to use the UnderSamplingClassifier that is referenced in the documentation

## Imports

In [2]:
import sys
sys.path.append('..')
import numpy as np, pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from skopt.space import Real, Integer, Categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from nestedcvtraining.api import find_best_model
from nestedcvtraining.switch_case import SwitchCase
from nestedcvtraining.under_sampling_classifier import UnderSamplingClassifier
from skopt import gbrt_minimize
from collections import Counter

## Dataset

In [3]:
dataset = pd.read_csv(
    "https://raw.githubusercontent.com/JaimeArboleda/nestedcvtraining/master/datasets/haberman.csv", header=None
)
values = dataset.values
X, y = values[:, :-1], (values[:, -1] - 1).astype(int)


In [4]:
X.shape

(306, 3)

In [5]:
Counter(y)

Counter({0: 225, 1: 81})

In this case we have a binary classification problem with some (but not too much) imbalance. It's a good candidate for the `UnderSamplingClassifier`, which is suited specially for binary problems or for multiclass problems where there is only one class that is imbalanced (and the rest are more or less balanced). This happens because the `UnderSamplingClassifier` will train an ensemble of models by taking all examples from the less frequent class and a stratified sample from the rest of the classes. 

## Model

Let's compare only two pipelines: one with oversampling using `SMOTE` and other without oversampling but using `UnderSamplingClassifier`, and see what works better.  

In [10]:
clf = SwitchCase(
    cases=[
        (
            "option_1",
            ImbPipeline(
                [
                    ("resampler", SMOTE(k_neighbors=3)), 
                    ("model", RandomForestClassifier())
                ]
            )
        ),
        (
            "option_2",
            UnderSamplingClassifier(RandomForestClassifier())
        )
    ],
    switch="option_1"
)

search_space = [
    Categorical(["option_1", "option_2"], name="switch"),
    Categorical(["minority", "all"], name="option_1__resampler__sampling_strategy"),
    Integer(5, 15, name="option_1__model__max_depth"),
    Integer(5, 15, name="option_2__estimator__max_depth"),
]

In [11]:
best_model, best_params, report = find_best_model(
    X=X,
    y=y,
    model=clf,
    search_space=search_space,
    verbose=False,
    k_inner=8,
    k_outer=8,
    n_initial_points=5,
    n_calls=5,
    calibrate="only_best",
    calibrate_params={"method": "sigmoid"},
    optimizing_metric=make_scorer(roc_auc_score, average='weighted', multi_class='ovr', needs_proba=True),
    other_metrics={"acc": "accuracy"},
    skopt_func=gbrt_minimize
)

Looping over 0 outer fold
Looping over 1 outer fold
Looping over 2 outer fold
Looping over 3 outer fold
Looping over 4 outer fold
Looping over 5 outer fold
Looping over 6 outer fold
Looping over 7 outer fold


In [12]:
report.get_best_params()

{'switch': ['option_1',
  'option_1',
  'option_2',
  'option_1',
  'option_2',
  'option_1',
  'option_2',
  'option_2'],
 'option_1__resampler__sampling_strategy': ['minority',
  'minority',
  'all',
  'minority',
  'all',
  'minority',
  'minority',
  'minority'],
 'option_1__model__max_depth': [10, 14, 5, 9, 13, 11, 14, 9],
 'option_2__estimator__max_depth': [8, 6, 5, 12, 6, 8, 6, 5]}

In [13]:
df = report.to_dataframe()
df[df["best"]==True]

,best,outer_kfold,model,outer_test_indexes,param__option_1__model__max_depth,param__option_1__resampler__sampling_strategy,param__option_2__estimator__max_depth,param__switch,inner_validation_metrics__optimizing_metric,inner_validation_metrics__acc,outer_test_metrics__optimizing_metric,outer_test_metrics__acc
3,True,0,CalibratedClassifierCV(base_estimator=SwitchCa...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10,minority,8,option_1,0.704803,0.648619,0.601724,0.743590
9,True,1,CalibratedClassifierCV(base_estimator=SwitchCa...,"[32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 46, 4...",14,minority,6,option_1,0.659653,0.636586,0.275974,0.717949
10,True,2,CalibratedClassifierCV(base_estimator=SwitchCa...,"[70, 71, 72, 76, 77, 78, 79, 83, 84, 85, 86, 8...",5,all,5,option_2,0.682211,0.343026,0.603571,0.631579
19,True,3,CalibratedClassifierCV(base_estimator=SwitchCa...,"[116, 117, 118, 119, 120, 121, 122, 123, 124, ...",9,minority,12,option_1,0.618275,0.613302,0.783929,0.736842
21,True,4,CalibratedClassifierCV(base_estimator=SwitchCa...,"[152, 153, 154, 155, 158, 159, 160, 161, 162, ...",13,all,6,option_2,0.630185,0.287210,0.835714,0.710526
26,True,5,CalibratedClassifierCV(base_estimator=SwitchCa...,"[190, 191, 194, 195, 196, 197, 198, 199, 200, ...",11,minority,8,option_1,0.645174,0.594363,0.739286,0.736842
33,True,6,CalibratedClassifierCV(base_estimator=SwitchCa...,"[228, 231, 232, 233, 234, 235, 236, 237, 238, ...",14,minority,6,option_2,0.642338,0.283645,0.539286,0.710526
38,True,7,CalibratedClassifierCV(base_estimator=SwitchCa...,"[267, 269, 270, 271, 272, 273, 274, 275, 276, ...",9,minority,5,option_2,0.626817,0.257576,0.773214,0.736842


It looks like both methods, in this case, perform more or less equally well. Anyway, the results show a lot of variability, probably because we don't have too much samples (inner test set has only 5 elements!!). We should use a less folds to have greater tests sets. Let's do that. 

In [15]:
best_model, best_params, report = find_best_model(
    X=X,
    y=y,
    model=clf,
    search_space=search_space,
    verbose=False,
    k_inner=3,
    k_outer=3,
    n_initial_points=5,
    n_calls=5,
    calibrate="only_best",
    calibrate_params={"method": "sigmoid"},
    optimizing_metric=make_scorer(roc_auc_score, average='weighted', multi_class='ovr', needs_proba=True),
    other_metrics={"acc": "accuracy"},
    skopt_func=gbrt_minimize
)

Looping over 0 outer fold
Looping over 1 outer fold
Looping over 2 outer fold


In [16]:
report.get_best_params()

{'switch': ['option_2', 'option_1', 'option_2'],
 'option_1__resampler__sampling_strategy': ['all', 'all', 'all'],
 'option_1__model__max_depth': [13, 9, 5],
 'option_2__estimator__max_depth': [12, 6, 11]}

In [17]:
df = report.to_dataframe()
df[df["best"]==True]

,best,outer_kfold,model,outer_test_indexes,param__option_1__model__max_depth,param__option_1__resampler__sampling_strategy,param__option_2__estimator__max_depth,param__switch,inner_validation_metrics__optimizing_metric,inner_validation_metrics__acc,outer_test_metrics__optimizing_metric,outer_test_metrics__acc
4,True,0,CalibratedClassifierCV(base_estimator=SwitchCa...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",13,all,12,option_2,0.665370,0.421569,0.576049,0.705882
6,True,1,CalibratedClassifierCV(base_estimator=SwitchCa...,"[101, 102, 103, 104, 105, 106, 108, 109, 110, ...",9,all,6,option_1,0.548333,0.573529,0.579012,0.725490
10,True,2,CalibratedClassifierCV(base_estimator=SwitchCa...,"[198, 199, 206, 207, 208, 209, 210, 211, 212, ...",5,all,11,option_2,0.634815,0.343137,0.662716,0.735294
